In [10]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_openai import ChatOpenAI 
import os
from dotenv import load_dotenv

In [11]:
load_dotenv()
llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model="gpt-3.5-turbo",temperature=0)

def resposta_simples(query):
    return f"A resposta para {query} é Paris"

In [12]:
tools = [
    Tool(
        name= "Respostas Simples",
        func= resposta_simples,
        description="Uma ferramenta simples para responder perguntas diretas"
    )
]

In [13]:
agent= initialize_agent(tools,llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

input_query ="Qual é a capital da França?"

response= agent.run(input_query)
print(response)



> Entering new AgentExecutor chain...
I should use the tool Respostas Simples to quickly find the answer.
Action: Respostas Simples
Action Input: Qual é a capital da França?
Observation: A resposta para Qual é a capital da França? é Paris
Thought:Eu agora sei a resposta final
Final Answer: A capital da França é Paris.

> Finished chain.
A capital da França é Paris.


MODELO AVANÇADO DE AGENTE ESPECIALIZADO


In [14]:
from langchain.agents import AgentExecutor
import sqlite3

In [16]:
def create_database():
    conn = sqlite3.connect(":memory:")
    c= conn.cursor()
    c.execute("""CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXST, age INTEGER)""")
    c.execute("INSERT INTO users(name, age) VALUES('João',23),('Maria',30)")
    conn.commit()
    return conn

In [25]:
def search_user_db(user_name):
    conn= create_database()
    c = conn.cursor()
    c.execute("SELECT name, age FROM users WHERE name = ?",(user_name,))
    result = c.fetchone()
    conn.close()
    if result:
        return f"{result[0]} tem {result[1]} anos"
    else: 
        return "Usuário não encontrado"

def sum_numbers(numbers):
    if isinstance(numbers,str):
        numbers= eval(numbers)
    if isinstance(numbers, list):
        total= sum(numbers)
        return f"A soma dos números{numbers} é {total}"
    else: 
        return "Erro: A entrada não é uma lista de números válida"

In [26]:
tools=[
    Tool(
        name="Buscar Usuário no BD",
        func=search_user_db,
        description="Procura informações sobre um usuaŕio na base de dados"
    ),
    Tool(
        name="Calcular soma",
        func=sum_numbers,
        description="Calcula a soma de dois numeros"
    )
]

In [27]:
agent= initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [28]:
input_query="Qual é a soma de 5, 25 e 15? Quantos anos Maria tem?"
response= agent.run(input_query)
print(response)



> Entering new AgentExecutor chain...
I need to calculate the sum of 5, 25, and 15 first. Then I need to find out Maria's age.
Action: Calcular soma
Action Input: 5, 25, 15
Observation: Erro: A entrada não é uma lista de números válida
Thought:I need to input the numbers as a list.
Action: Calcular soma
Action Input: [5, 25, 15]
Observation: A soma dos números[5, 25, 15] é 45
Thought:Now I need to find out Maria's age.
Action: Buscar Usuário no BD
Action Input: Maria
Observation: Maria tem 30 anos
Thought:I now know the final answer
Final Answer: A soma de 5, 25 e 15 é 45. Maria tem 30 anos.

> Finished chain.
A soma de 5, 25 e 15 é 45. Maria tem 30 anos.
